In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [2]:
# The following lines adjust the granularity of reporting.
#pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir('/tf')
PGUSER = os.getenv('POSTGRES_USER')
PGPW = os.getenv('POSTGRES_PW')
PGDB = os.getenv('POSTGRES_DB')
NULLFLAG = -9999
ONEDAY = 24 * 60 * 60
WEEK = 7 * ONEDAY

In [3]:
def getGPDfromPD(df: pd.DataFrame, geomCol: str, crs: str = "EPSG:3978") -> gpd.GeoDataFrame:
    """
    Convert a pandas dataframe to a geopandas dataframe
    :param df: pandas dataframe
    :param geomCol: name of the geometry column
    :param crs: coordinate reference system
    :return: geopandas dataframe
    """
    if 'geom' in df.columns:
        df.rename(columns={'geom': 'geometry'}, inplace=True)

    df[geomCol] = df[geomCol].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry=geomCol, crs=crs)
    return gdf

In [4]:
# load fire 
fires = "Data/GEOlgFireFifty.csv"
dfTemp = pd.read_csv(fires)

dfFire = getGPDfromPD(dfTemp, 'geometry')
print(dfFire.columns)

Index(['EntryID', 'FIRE_ID', 'FIRENAME', 'YEAR', 'MONTH', 'DAY', 'REP_DATE',
       'OUT_DATE', 'DECADE', 'SIZE_HA', 'CALC_HA', 'CAUSE', 'CFS_REF_ID',
       'geometry'],
      dtype='object')


In [5]:
# first read in the csv file into pd
dfTemp = pd.read_csv('Data/GEOProvincialBoundaries.csv')

dfProvinces = getGPDfromPD(dfTemp, 'geometry')

In [6]:
dfSK = dfFire
# keep only dfNearest where geomtry is containe din dfProvinces Saskatchewan
dfSK['keep'] = False
for index, row in dfSK.iterrows():
    if dfProvinces[dfProvinces['provID'] == 'SK'].contains(row['geometry']).any():
        dfSK.at[index, 'keep'] = True


In [7]:
dfSK = dfSK[dfSK['keep'] == True]

In [8]:
# select unique years
years = dfSK['YEAR'].unique()

# count rows with year
for year in years:
    print(year, len(dfSK[dfSK['YEAR'] == year]))



2008 58
1981 64
1990 24
1991 18
2004 61
2015 112
2005 45
2006 73
1992 8
1994 36
2009 25
1985 21
1996 9
2007 24
1998 86
2003 35
2010 86
1999 35
2011 23
2012 44
2013 33
1984 37
1982 20
1983 20
1986 6
1989 94
1987 33
1988 19
1995 40
1993 23
1997 5
2000 20
2001 30
2002 66
2016 25
2014 48
2017 47
2018 36
2019 15
2020 4
